## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Apr. 22nd Apr. 29th


In [3]:
filename = glob.glob('redfin*.csv')
filename = filename[0]

In [4]:
df = pd.read_csv(filename)
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [6]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

155


In [7]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$789,242


In [8]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$545


In [9]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$122,332,463


## Previous Week's Values

In [10]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
160
Input Previous Week Condo Average Sales Price:
977219
Input Previous Week Condo Average PSF:
651
Input Previous Week Condo Sales total (ex: 198_000_000)
156000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [11]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [12]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
pd.set_option('display.max_columns',None)

In [15]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
43,PAST SALE,April-26-2024,Condo/Co-op,360 Ocean Dr Unit 601S,Key Biscayne,FL,33149.0,9600000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,3791.0,NaN,2014.0,NaN,2532.0,5732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11549038,N,Y,25.694584,-80.157715,1,orange


In [16]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [17]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [19]:
m.save('index.html')

## Data snagger

In [20]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [21]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
43,PAST SALE,April-26-2024,Condo/Co-op,360 Ocean Dr Unit 601S,Key Biscayne,FL,33149.0,9600000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,3791.0,NaN,2014.0,NaN,2532.0,5732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11549038,N,Y,25.694584,-80.157715,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [22]:
df_top_ten['PRICE'].iloc[0]

9600000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [23]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_042924


# CREATE TEMPLATE 

In [24]:
muni_set = set(df_top_ten['CITY'])

In [25]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [26]:
df_top_ten.reset_index(inplace=True,drop=True)

In [27]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [28]:
top_sale

'OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit 601S in Key Biscayne'

In [29]:
# previous_week_condo_sales_total = 130_000_000

In [30]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [31]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANA KEY BISCAYNE CONDO tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,700,000M to $9,600,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Miami Beach, Aventura, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 155 condo sales totaling $122,332,463 million from Apr. 22nd to Apr. 29th. The previous week, brokers closed 160 condo sales totaling $156,000,000.

Last week’s units sold for an average of $789,242, lower than the $977,219 sales average from the previous week. The average pr

In [32]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [33]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [34]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: OCEANA KEY BISCAYNE CONDO tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,700,000M to $9,600,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Miami Beach, Aventura, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 155 condo sales totaling $122,332,463 million from Apr. 22nd to Apr. 29th. The previous week, brokers closed 160 condo sales totaling $156,000,000.

Last week’s units sold for an average of $789,242, lower than the $977,219 sales average from the previous week. The average pr

In [38]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,April-10-2024,Condo/Co-op,5775 Collins Ave Ph -2,Miami Beach,FL,33140.0,25000000.0,6.0,7.0,AMD PL OF 2ND OCEAN FRONT,6461.0,NaN,2021.0,NaN,3869.0,16706.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11478573,N,Y,25.839257,-80.120395,1,orange,AMD PL OF 2ND OCEAN FRONT at 5775 Collins Ave ...
1,PAST SALE,April-10-2024,Condo/Co-op,350 Ocean Dr Unit 1002N,Key Biscayne,FL,33149.0,8300000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,3369.0,NaN,2014.0,NaN,2464.0,5101.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11469496,N,Y,25.694534,-80.157613,2,blue,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
2,PAST SALE,April-8-2024,Condo/Co-op,4775 Collins Ave Unit TS4404,Miami Beach,FL,33140.0,7325000.0,3.0,4.5,GREEN DIAMOND CONDO,2670.0,NaN,2000.0,NaN,2743.0,3356.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4775-Col...,MARMLS,A11474820,N,Y,25.822419,-80.121616,3,blue,GREEN DIAMOND CONDO at 4775 Collins Ave Unit T...
3,PAST SALE,April-11-2024,Condo/Co-op,16901 Collins Ave #2805,Sunny Isles Beach,FL,33160.0,4500000.0,4.0,4.5,JADE SIGNATURE CONDO,3292.0,NaN,2018.0,NaN,1367.0,5290.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11472728,N,Y,25.932816,-80.121161,4,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #280...
4,PAST SALE,April-9-2024,Condo/Co-op,700 NE 25th St Ph 2,Miami,FL,33137.0,4500000.0,4.0,4.5,STAR LOFTS ON THE BAY CON,6291.0,NaN,2007.0,NaN,715.0,4341.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-25th-St...,MARMLS,A11451260,N,Y,25.800829,-80.185785,5,blue,STAR LOFTS ON THE BAY CON at 700 NE 25th St Ph...
5,PAST SALE,April-12-2024,Condo/Co-op,16047 Collins Ave #3303,Sunny Isles Beach,FL,33160.0,3500000.0,4.0,3.5,TURNBERRY OCEAN COLONY SO,3480.0,NaN,2007.0,NaN,1006.0,5744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11450930,N,Y,25.925401,-80.121609,6,blue,TURNBERRY OCEAN COLONY SO at 16047 Collins Ave...
6,PAST SALE,April-8-2024,Condo/Co-op,4779 Collins Ave Ph 4301,Miami Beach,FL,33140.0,3500000.0,3.0,3.0,BLUE DIAMOND CONDO,1940.0,NaN,2000.0,NaN,1804.0,2737.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4779-Col...,MARMLS,A11442303,N,Y,25.823220,-80.121531,7,blue,BLUE DIAMOND CONDO at 4779 Collins Ave Ph 4301...
7,PAST SALE,April-8-2024,Condo/Co-op,16047 Collins Ave #1001,Sunny Isles Beach,FL,33160.0,3375000.0,3.0,4.5,TURNBERRY OCEAN COLONY,2772.0,NaN,2007.0,NaN,1218.0,5417.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11521244,N,Y,25.925401,-80.121609,8,blue,TURNBERRY OCEAN COLONY at 16047 Collins Ave #1...
8,PAST SALE,April-9-2024,Condo/Co-op,9999 Collins Ave Unit 14E,Bal Harbour,FL,33154.0,3175000.0,2.0,5.0,BAL HARBOUR TOWER CONDO,3230.0,NaN,1990.0,NaN,983.0,3923.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11502043,N,Y,25.891954,-80.122707,9,blue,BAL HARBOUR TOWER CONDO at 9999 Collins Ave Un...
9,PAST SALE,April-11-2024,Condo/Co-op,100 S Pointe Dr #702,Miami Beach,FL,33139.0,3050000.0,1.0,1.5,CONTINUUM ON SOUTH BEACH,1201.0,NaN,2002.0,NaN,2540.0,3052.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11502799,N,Y,25.767177,-80.132816,10,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #7...


In [35]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-601S/home/79133310


In [36]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-4703/home/56743878


In [48]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-205S/home/79133306


In [45]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-601S/home/79133310


In [46]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/North-Miami-Beach/15811-Collins-Ave-33160/unit-801/home/43462401


In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,April-26-2024,Condo/Co-op,360 Ocean Dr Unit 601S,Key Biscayne,FL,33149.0,9600000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,3791.0,NaN,2014.0,NaN,2532.0,5732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11549038,N,Y,25.694584,-80.157715,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,April-26-2024,Condo/Co-op,16901 Collins Ave #4703,Sunny Isles Beach,FL,33160.0,4800000.0,3.0,4.5,JADE SIGNATURE CONDO,2915.0,NaN,2018.0,NaN,1647.0,5099.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11356790,N,Y,25.932756,-80.121005,2,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #470...
2,PAST SALE,April-22-2024,Condo/Co-op,5959 Collins Ave #1802,Miami Beach,FL,33140.0,4370000.0,3.0,4.5,THE RESIDENCES AT THE BAT,2572.0,NaN,2005.0,NaN,1699.0,9400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11453106,N,Y,25.841880,-80.120969,3,blue,THE RESIDENCES AT THE BAT at 5959 Collins Ave ...
3,PAST SALE,April-22-2024,Condo/Co-op,16047 Collins Ave #1703,Sunny Isles Beach,FL,33160.0,4050000.0,3.0,5.5,TURNBERRY OCEAN COLONY SO,3480.0,NaN,2007.0,NaN,1164.0,6103.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11574683,N,Y,25.925401,-80.121609,4,blue,TURNBERRY OCEAN COLONY SO at 16047 Collins Ave...
4,PAST SALE,April-25-2024,Condo/Co-op,300 S Pointe Dr #2001,Miami Beach,FL,33139.0,3550000.0,3.0,2.5,PORTOFINO TOWER CONDO,2340.0,NaN,1996.0,NaN,1517.0,4015.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11484763,N,Y,25.767706,-80.134877,5,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #2001...
5,PAST SALE,April-24-2024,Condo/Co-op,3300 NE 188th St Unit UPH16,Aventura,FL,33180.0,3050000.0,4.0,5.5,ECHO CONDO,2942.0,NaN,2015.0,NaN,1037.0,3272.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11467239,N,Y,25.948955,-80.135269,6,blue,ECHO CONDO at 3300 NE 188th St Unit UPH16 in A...
6,PAST SALE,April-26-2024,Condo/Co-op,6799 Collins Ave #205,Miami Beach,FL,33141.0,2950625.0,3.0,3.0,SOUTH CARILLON BEACH,1956.0,NaN,2008.0,NaN,1508.0,5852.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6799-Col...,MARMLS,A11471479,N,Y,25.852725,-80.120046,7,blue,SOUTH CARILLON BEACH at 6799 Collins Ave #205 ...
7,PAST SALE,April-25-2024,Condo/Co-op,300 S Pointe Dr #2306,Miami Beach,FL,33139.0,2900000.0,3.0,2.5,PORTOFINO TOWER CONDO,2040.0,NaN,1996.0,NaN,1422.0,2950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11451252,N,Y,25.767706,-80.134877,8,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #2306...
8,PAST SALE,April-25-2024,Condo/Co-op,360 Ocean Dr Unit 205S,Key Biscayne,FL,33149.0,2700000.0,2.0,3.5,OCEANA KEY BISCAYNE CONDO,1830.0,NaN,2014.0,NaN,1475.0,2818.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11369459,N,Y,25.694584,-80.157715,9,blue,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
9,PAST SALE,April-23-2024,Condo/Co-op,15811 Collins Ave #801,Sunny Isles Beach,FL,33160.0,2700000.0,3.0,3.5,TDR TOWER III CONDO,2928.0,NaN,2010.0,NaN,922.0,2706.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11431859,N,Y,25.922834,-80.121864,10,blue,TDR TOWER III CONDO at 15811 Collins Ave #801 ...


## Time on Market Calculator

In [42]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 3, 14) ## List (Earlier) date
date2 = datetime(2024, 4, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

43
